In [1]:
# Import libraries
import pandas as pd
import seaborn
import json
%matplotlib inline
import requests
from bs4 import BeautifulSoup

# Scraping

#### We will start by using postman to spy on the get requests that are sent for both website, in order to retrieve the url that loads the rankings in the corresponding websites. To do this, we send requests to both websites and retrieve the information as json files and extract two dictionarries that will be used to form the dataframes

In [2]:
r_THE = requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
r_QS = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508593854921')

data_THE = r_THE.json() #ranking data for timeshighereducation.com
data_QS = r_QS.json()  #ranking data for topuniversities.com

##### We only took the data part for each json, which is of interest

In [3]:
print(list(data_THE.keys()))
print(list(data_QS.keys()))

['pillars', 'locations', 'data', 'subjects']
['data']


In [4]:
df_rank_THE = pd.DataFrame(data_THE['data'])
df_rank_QS = pd.DataFrame(data_QS['data'])

##### We create the two dataframes and drop all the universites ranked above 200

In [5]:
# We need to clean the rankings in replacing first special characters

df_rank_THE['rank'] = df_rank_THE['rank'].replace("=|–|+", "", regex=True).astype(int)
df_rank_THE = df_rank_THE.drop(df_rank_THE[df_rank_THE['rank'] > 200].index)

df_rank_QS['rank_display'] = df_rank_QS['rank_display'].replace("=|-|+", "", regex=True).astype(int)
df_rank_QS = df_rank_QS.drop(df_rank_QS[df_rank_QS['rank_display'] > 200].index)

##### Then, we select the required features for both dataframes

In [6]:
THE_features = ['name', 'location', 'rank', 'stats_number_students', 'stats_pc_intl_students', 'stats_student_staff_ratio']
df_rank_THE = df_rank_THE[THE_features]

df_rank_THE = df_rank_THE.set_index('name').sort_index(ascending=True)


In [7]:
QS_features = ['country', 'region', 'rank_display']

df_rank_QS = df_rank_QS.set_index('title').sort_index(ascending=True)

urls = df_rank_QS['url'] # We need to search further informations as the list isn't exhaustive, hence the url we are saving 
df_rank_QS = df_rank_QS[QS_features]

# Question 1

##### For the first webpage, we need to scrap more data from the detail page of each university. Postman was very helpful for finding tags with the corresponing informations.

In [8]:
link = 'https://www.topuniversities.com/'
university = {'title': [],
              'faculty members total': [],
              'faculty members international': [],
              'students total': [],
              'students international': []}

for uni in urls.iteritems():
    soup = BeautifulSoup(requests.get(link + uni[1]).text, 'html.parser')
    
    faculty = soup.find('div', class_='faculty-main wrapper col-md-4 item active')
    if faculty is not None:
        faculty = faculty.find_all('div', class_='number')
        
    students = soup.find('div', class_='students-main wrapper col-md-4 item')
    if students is not None:
        students = students.find('div', class_='number')
    
    int_students = soup.find('div', class_='int-students-main wrapper col-md-4 item')
    if int_students is not None:
        int_students = int_students.find('div', class_='number')
    
    
    if faculty is not None:
        if len(faculty) > 1:
            university['faculty members total'].append(int(faculty[0].text.replace(',', '')))
            university['faculty members international'].append(int(faculty[1].text.replace(',', '')))
        else:
            university['faculty members total'].append(int(faculty[0].text.replace(',', '')))
            university['faculty members international'].append(None)
    else:
        university['faculty members total'].append(None)
        university['faculty members international'].append(None)
        
        
    if students is not None:
        university['students total'].append(int(students.text.replace(',', '')))
        
    else:
        university['students total'].append(None)
    
    if int_students is not None:
        university['students international'].append(int(int_students.text.replace(',', '')))
        
    else:
        university['students international'].append(None)

    university['title'].append(uni[0])
        

##### We join the original dataframe with the additional informations collected and rearrange headers in a intuitive way

In [9]:
df_rank_QS_supp = pd.DataFrame(university).set_index('title')
df_rank_QS = df_rank_QS.join(df_rank_QS_supp)

In [10]:
col = [('country', None), ('region', None), ('rank', None), ('faculty members', 'international'), ('faculty members', 'total'), ('students', 'international'), ('students', 'total')]
df_rank_QS.columns = pd.MultiIndex.from_tuples([('', x[0]) if pd.isnull(x[1]) else x for x in col])
df_rank_QS.index.name = 'name'

## University

In [31]:
QS_ratio_university = df_rank_QS.copy()[['faculty members', 'students']]

QS_ratio_university['ratio', 'faculty/students'] = QS_ratio_university['faculty members', 'total'] / QS_ratio_university['students', 'total']
QS_ratio_university['ratio', 'international/total'] = QS_ratio_university['students', 'international'] / QS_ratio_university['students', 'total']

### Best faculty/students ratio

In [32]:
QS_ratio_university.sort_values(('ratio', 'faculty/students'), ascending=False).head()

faculty members          \
                                               international   total   
name                                                                   
California Institute of Technology (Caltech)           350.0   953.0   
Yale University                                       1708.0  4940.0   
University of Oxford                                  2964.0  6750.0   
University of Cambridge                               2278.0  5490.0   
Johns Hopkins University                              1061.0  4462.0   

                                                  students           \
                                             international    total   
name                                                                  
California Institute of Technology (Caltech)         647.0   2255.0   
Yale University                                     2469.0  12402.0   
University of Oxford                                7353.0  19720.0   
University of Cambridge                             6699.0  18770.0   
Johns Hopkins University                            4105.0  16146.0   

                                                        ratio  \
                                             faculty/students   
name                                                            
California Institute of Technology (Caltech)         0.422616   
Yale University                                      0.398323   
University of Oxford                                 0.342292   
University of Cambridge                              0.292488   
Johns Hopkins University                             0.276353   

                                                                  
                                             international/total  
name                                                              
California Institute of Technology (Caltech)            0.286918  
Yale University                                         0.199081  
University of Oxford                                    0.372870  
University of Cambridge                                 0.356899  
Johns Hopkins University                                0.254243

### Best international ratio

In [13]:
QS_ratio_university.sort_values(('ratio', 'international/total'), ascending=False).head()

faculty members          \
                                                     international   total   
name                                                                         
London School of Economics and Political Scienc...           687.0  1088.0   
Ecole Polytechnique Fédérale de Lausanne (EPFL)             1300.0  1695.0   
Imperial College London                                     2071.0  3930.0   
Maastricht University                                        502.0  1277.0   
Carnegie Mellon University                                   425.0  1342.0   

                                                        students           \
                                                   international    total   
name                                                                        
London School of Economics and Political Scienc...        6748.0   9760.0   
Ecole Polytechnique Fédérale de Lausanne (EPFL)           5896.0  10343.0   
Imperial College London                                   8746.0  16090.0   
Maastricht University                                     8234.0  16385.0   
Carnegie Mellon University                                6385.0  13356.0   

                                                              ratio  \
                                                   faculty/students   
name                                                                  
London School of Economics and Political Scienc...         0.111475   
Ecole Polytechnique Fédérale de Lausanne (EPFL)            0.163879   
Imperial College London                                    0.244251   
Maastricht University                                      0.077937   
Carnegie Mellon University                                 0.100479   

                                                                        
                                                   international/total  
name                                                                    
London School of Economics and Political Scienc...            0.691393  
Ecole Polytechnique Fédérale de Lausanne (EPFL)               0.570047  
Imperial College London                                       0.543567  
Maastricht University                                         0.502533  
Carnegie Mellon University                                    0.478062

## Country

In [33]:
QS_ratio_country = df_rank_QS.copy()
QS_ratio_country = QS_ratio_country.groupby([('', 'country')]).sum()[['faculty members', 'students']]
QS_ratio_country.index.name = 'Country'

QS_ratio_country['ratio', 'faculty/students'] = QS_ratio_country['faculty members', 'total']/QS_ratio_country['students', 'total']
QS_ratio_country['ratio', 'international/total'] = QS_ratio_country['students', 'international']/QS_ratio_country['students', 'total']

### Best faculty/student ratio

In [34]:
QS_ratio_country.sort_values(('ratio', 'faculty/students'), ascending=False).head()

faculty members               students                     ratio  \
               international    total international    total faculty/students   
Country                                                                         
Russia                 373.0   6709.0        5098.0  30233.0         0.221910   
Denmark               3904.0  11916.0        9543.0  67223.0         0.177261   
Saudi Arabia           665.0   1062.0         989.0   6040.0         0.175828   
Singapore             6079.0   9444.0       16168.0  58466.0         0.161530   
Malaysia               655.0   2755.0        3476.0  17902.0         0.153893   

                                  
             international/total  
Country                           
Russia                  0.168624  
Denmark                 0.141960  
Saudi Arabia            0.163742  
Singapore               0.276537  
Malaysia                0.194168

### Best international ratio

In [35]:
QS_ratio_country.sort_values(('ratio', 'international/total'), ascending=False).head()

faculty members               students            \
                 international    total international     total   
Country                                                           
Australia              11382.0  22034.0      106359.0  301994.0   
United Kingdom         30216.0  79934.0      199426.0  583621.0   
Hong Kong               6296.0  10166.0       24499.0   78838.0   
Austria                 1572.0   4117.0       19667.0   63446.0   
Switzerland             9208.0  15323.0       32995.0  109112.0   

                          ratio                      
               faculty/students international/total  
Country                                              
Australia              0.072962            0.352189  
United Kingdom         0.136962            0.341705  
Hong Kong              0.128948            0.310751  
Austria                0.064890            0.309980  
Switzerland            0.140434            0.302396

## Region

In [36]:
QS_ratio_region = df_rank_QS.copy()
QS_ratio_region = QS_ratio_region.groupby([('', 'region')]).sum()[['faculty members', 'students']]
QS_ratio_region.index.name = 'Region'

QS_ratio_region['ratio', 'faculty/students'] = QS_ratio_region['faculty members', 'total']/QS_ratio_region['students', 'total']
QS_ratio_region['ratio', 'international/total'] = QS_ratio_region['students', 'international']/QS_ratio_region['students', 'total']

### Best faculty/student ratio

In [37]:
QS_ratio_region.sort_values(('ratio', 'faculty/students'), ascending=False).head()

faculty members                students             \
                international     total international      total   
Region                                                             
Asia                  25462.0  106734.0      110100.0   807003.0   
North America         43836.0  182123.0      292116.0  1546353.0   
Europe                67598.0  218358.0      449364.0  1957251.0   
Latin America          5521.0   43126.0       34737.0   396902.0   
Africa                  379.0    1733.0        3325.0    19593.0   

                         ratio                      
              faculty/students international/total  
Region                                              
Asia                  0.132260            0.136431  
North America         0.117776            0.188906  
Europe                0.111564            0.229589  
Latin America         0.108657            0.087520  
Africa                0.088450            0.169703

### Best international ratio

In [38]:
QS_ratio_region.sort_values(('ratio', 'international/total'), ascending=False).head()

faculty members                students             \
                international     total international      total   
Region                                                             
Oceania               12786.0   25347.0      118798.0   350167.0   
Europe                67598.0  218358.0      449364.0  1957251.0   
North America         43836.0  182123.0      292116.0  1546353.0   
Africa                  379.0    1733.0        3325.0    19593.0   
Asia                  25462.0  106734.0      110100.0   807003.0   

                         ratio                      
              faculty/students international/total  
Region                                              
Oceania               0.072385            0.339261  
Europe                0.111564            0.229589  
North America         0.117776            0.188906  
Africa                0.088450            0.169703  
Asia                  0.132260            0.136431

# Question 2

In [20]:
df_rank_THE['stats_number_students'] = df_rank_THE.stats_number_students.apply(lambda x: x.replace(',', ''))
df_rank_THE['stats_pc_intl_students'] = df_rank_THE.stats_pc_intl_students.apply(lambda x: float(x.replace('%', ''))/100)
df_rank_THE['stats_student_staff_ratio'] = df_rank_THE.stats_student_staff_ratio.apply(lambda x: 1/float(x))

In [21]:
col = [('', 'Country'),('', 'rank'), ('students', 'total'), ('ratio', 'international/total'), ('ratio', 'faculty/students')]

df_rank_THE.columns = pd.MultiIndex.from_tuples([('', x[0]) if pd.isnull(x[1]) else x for x in col])

df_rank_THE['students', 'international'] = df_rank_THE[('students', 'total')].apply(lambda x: float(x)).multiply(df_rank_THE[('ratio', 'international/total')])
df_rank_THE['faculty', 'total'] = df_rank_THE[('ratio', 'faculty/students')].apply(lambda x: float(x)).multiply(df_rank_THE[('students', 'total')].apply(lambda x: float(x)))

df_rank_THE['faculty', 'total'] = df_rank_THE['faculty', 'total'].apply(lambda x: int(x))
df_rank_THE['students', 'total'] = df_rank_THE['students', 'total'].apply(lambda x: int(x))

df_rank_THE = df_rank_THE[[('', 'Country'), ('', 'rank'), ('faculty', 'total'), ('students', 'international'), ('students', 'total'), ('ratio', 'faculty/students'), ('ratio', 'international/total')]]

In [22]:
THE_ratio_university = df_rank_THE.copy()[['students', 'ratio']]

## University

### Best faculty/student ratio

In [23]:
THE_ratio_university.sort_values(('ratio', 'faculty/students'), ascending=False).head()

students                   ratio  \
                         international  total faculty/students   
name                                                             
Vanderbilt University          1561.43  12011         0.303030   
University of Copenhagen       4255.30  30395         0.243902   
Johns Hopkins University       3719.52  15498         0.232558   
Yale University                2552.55  12155         0.232558   
University of Rochester        2794.44   9636         0.232558   

                                              
                         international/total  
name                                          
Vanderbilt University                   0.13  
University of Copenhagen                0.14  
Johns Hopkins University                0.24  
Yale University                         0.21  
University of Rochester                 0.29

### Best international ratio

In [24]:
THE_ratio_university.sort_values(('ratio', 'international/total'), ascending=False).head()

students         \
                                                 international  total   
name                                                                    
London School of Economics and Political Science       7146.15  10065   
University of Luxembourg                               2832.33   4969   
Imperial College London                                8721.35  15857   
École Polytechnique Fédérale de Lausanne               5460.40   9928   
Maastricht University                                  8363.50  16727   

                                                            ratio  \
                                                 faculty/students   
name                                                                
London School of Economics and Political Science         0.081967   
University of Luxembourg                                 0.068493   
Imperial College London                                  0.087719   
École Polytechnique Fédérale de Lausanne                 0.089286   
Maastricht University                                    0.055556   

                                                                      
                                                 international/total  
name                                                                  
London School of Economics and Political Science                0.71  
University of Luxembourg                                        0.57  
Imperial College London                                         0.55  
École Polytechnique Fédérale de Lausanne                        0.55  
Maastricht University                                           0.50

## Country

In [39]:
THE_ratio_country = df_rank_THE.copy()
THE_ratio_country = THE_ratio_country.groupby([('', 'Country')]).sum()[['faculty', 'students']]
THE_ratio_country.index.name = 'Country'

THE_ratio_country['ratio', 'faculty/students'] = THE_ratio_country['faculty', 'total']/THE_ratio_country['students', 'total']
THE_ratio_country['ratio', 'international/total'] = THE_ratio_country['students', 'international']/THE_ratio_country['students', 'total']

### Best faculty/student ratio

In [40]:
THE_ratio_country.sort_values(('ratio', 'faculty/students'), ascending=False).head()

faculty      students                    ratio  \
                     total international   total faculty/students   
Country                                                             
Denmark              10596       9326.86   64110         0.165278   
Russian Federation    4004       6431.92   29236         0.136954   
Japan                 6464       4398.48   48481         0.133331   
Italy                  157        132.65    1205         0.130290   
Switzerland          10048      32745.97  107852         0.093165   

                                        
                   international/total  
Country                                 
Denmark                       0.145482  
Russian Federation            0.220000  
Japan                         0.090726  
Italy                         0.110083  
Switzerland                   0.303619

### Best international ratio

In [41]:
THE_ratio_country.sort_values(('ratio', 'international/total'), ascending=False)

faculty      students                     ratio  \
                     total international    total faculty/students   
Country                                                              
Luxembourg             340       2832.33     4969         0.068424   
United Kingdom       44425     213059.74   596449         0.074482   
Hong Kong             4140      25158.79    77663         0.053307   
Australia             9937      83811.65   268630         0.036991   
Singapore             3364      17085.29    56101         0.059963   
Switzerland          10048      32745.97   107852         0.093165   
New Zealand           1614       8800.92    30348         0.053183   
Ireland                708       4362.39    16157         0.043820   
Austria               1700       9197.50    35375         0.048057   
Canada               13236      55905.59   249401         0.053071   
Russian Federation    4004       6431.92    29236         0.136954   
France                7435      18975.93    99378         0.074815   
Norway                1490       5268.70    27730         0.053732   
Belgium               3426      23768.44   130115         0.026331   
South Africa          1775       3739.50    20775         0.085439   
Netherlands          13599      41565.42   240180         0.056620   
United States       152317     280364.88  1705533         0.089308   
Spain                 3041       6510.08    43481         0.069939   
Germany              15346      87491.19   586364         0.026171   
Denmark              10596       9326.86    64110         0.165278   
Sweden                8481      16009.07   125746         0.067445   
Finland               2020       3900.72    35276         0.057263   
Italy                  157        132.65     1205         0.130290   
South Korea           4511       6771.90    63035         0.071563   
Japan                 6464       4398.48    48481         0.133331   
China                21388      22882.57   253911         0.084234   
Taiwan                2761       2540.64    31758         0.086939   

                                        
                   international/total  
Country                                 
Luxembourg                    0.570000  
United Kingdom                0.357214  
Hong Kong                     0.323948  
Australia                     0.311997  
Singapore                     0.304545  
Switzerland                   0.303619  
New Zealand                   0.290000  
Ireland                       0.270000  
Austria                       0.260000  
Canada                        0.224159  
Russian Federation            0.220000  
France                        0.190947  
Norway                        0.190000  
Belgium                       0.182673  
South Africa                  0.180000  
Netherlands                   0.173059  
United States                 0.164385  
Spain                         0.149722  
Germany                       0.149210  
Denmark                       0.145482  
Sweden                        0.127313  
Finland                       0.110577  
Italy                         0.110083  
South Korea                   0.107431  
Japan                         0.090726  
China                         0.090120  
Taiwan                        0.080000

# Question 3